In [1]:
from pyspark import SparkConf, SparkContext
#스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_03_reduce")
spark = SparkContext(conf=conf).getOrCreate()
spark

24/12/06 18:06:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local appName=241205_03_reduce>

In [2]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
from operator import add
sample_rdd.reduce(add)

15

# partition 이 나누어져 있을 때
```
lambda x,y : (x*2) + y
```

1. 1=1, y=2 >> (1*2) + 2 = 4
2. x=4, y=3 >> (4*2) + 3 = 11
3. x = 11, y=4 >> (11*2) + 4 = 26

In [4]:
#파티션이 1개일 때
sample_rdd1 = spark.parallelize([1,2,3,4])
sample_rdd1.reduce(lambda x,y: (x*2)+y)

26

In [5]:
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [6]:
#파티션 2개일 때 계산 결과 
sample_rdd2 = spark.parallelize([1,2,3,4],2)
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

In [7]:
sample_rdd2.reduce(lambda x,y: (x*2)+y)

18

## 파티션
1. 1=1, y=2 >> (1*2) + 2 = 4
2. x=3, y=4 >> (3*2) + 4 = 10
3. x = 4, y=10 >> (4*2) + 10 = 18

In [8]:
#파티션 3개일 때 계산 결과 
sample_rdd3 = spark.parallelize([1,2,3,4],3)
sample_rdd3.glom().collect()

[[1], [2], [3, 4]]

In [9]:
sample_rdd3.reduce(lambda x,y: (x*2)+y)

18

#### 파티션3 정리 

p1: 1 <br>
p2: 2 <br>
p3: x=3, y=4 > 10 <br>
<br>
p1, p2 결합 <br>
x=1, y=2 > 4 <br>

최종 <br>
x=4, y=10 > 18 <br>

# 리듀스 연산은 순서 의존적

각 파티션 내에서 연산은 독립적으로 실행, 최종 결과는 파티션 간의 결과가 결합

### fold 연산의 활용
```
fold(zeroValue, <func>)
```

zeroValue: 시작값, 항등원 *일때 1, +일 경우 0


In [10]:
sample_rdd4 = spark.parallelize([1, 2, 3, 4], 4)
sample_rdd4.glom().collect()

[[1], [2], [3], [4]]

In [11]:
sample_rdd4.reduce(lambda x,y: x*y)

24

- 시뮬레이션 <br>
p1: 1, p2: 2 = 2<br>
p3: 3, p4: 4 = 12<br>
2*12 = 24<br>

In [13]:
sample_rdd4.fold(1, lambda x,y:x*y)

24

In [14]:
#빈 RDD의 예외처리
sample_rdd5 = spark.parallelize([])
sample_rdd5.glom().collect()

[[]]

In [15]:
try:
    sample_rdd5.reduce(lambda x,y: x+y)
except ValueError as e:
    print(f'Reduce Error:{e}')

Reduce Error:Can not reduce() empty RDD


In [16]:
sample_rdd5.fold(2,lambda x,y: x+y)

4

## reduce(), fold() 비슷한 처리를 합니다. 

1. reduce() 는 단순한 RDD 축약 연산, 비어있지 않은 RDD 의 경우
2. fold()는 일반적이고 오류가 없는 연산, 초기값을 적용해서 비어있는 RDD, 파티션의 경우에도 적용할 수 있음.

In [17]:
rdd6 = spark.parallelize([1,"a",2])
rdd6.collect()

[1, 'a', 2]

In [18]:
try:
    print(rdd6.reduce(lambda x,y: str(x)+str(y)))
except Py4JJavaError as e :
    print(f'Reduce Error: {e}')

1a2


In [19]:
rdd6.fold("", lambda x,y: str(x)+str(y))

'1a2'

## groupby(), aggregate()


RDD.groupBy(lambda x:x%2).collect() <br>
RDD.aggregate(zeroValue,func1_partion, func2_combine)

In [20]:
# RDD 생성
rdd7 = spark.parallelize([1,2,1,1,3,5,5,8])
rdd7

ParallelCollectionRDD[22] at readRDDFromFile at PythonRDD.scala:274

In [21]:
 # groupBy 함수 적용
result = rdd7.groupBy(lambda x:x%2).collect() #각 요소를 2로 나눈 나머지를 기준으로 그룹화
result

[(1, <pyspark.resultiterable.ResultIterable at 0x7f578755bb50>),
 (0, <pyspark.resultiterable.ResultIterable at 0x7f577cc43850>)]

In [22]:
[(x,sorted(y)) for (x,y) in result]

[(1, [1, 1, 1, 3, 5, 5]), (0, [2, 8])]

In [23]:
# RDD.aggregate(zeroValue,func1_partion, func2_combine)
func1 = lambda x,y : (x[0] + y, x[1] +1)
func2 = lambda x,y : (x[0] + y[0], x[1] +y[1])

In [24]:
rdd8 = spark.parallelize([1,2,3,4], 2)
rdd8.glom().collect()

[[1, 2], [3, 4]]

In [25]:
rdd8.aggregate((0,0),func1, func2)

(10, 4)

In [ ]:
#simulation

p1: 파티션 1처리 [1,2]
1) 초기값: zeroValue = [0,0]
x=(0,0), y = 1 ([1,2] 에 1)

2) 첫번째 요소 1 처리 
func1(x, y) = (x[0] + y, x[1] + 1)
# x = (0, 0), y = 1
# 계산: (0 + 1, 0 + 1) → (1, 1)

3) 두번째 요소 2 처리 # 누적 (1,1)을 가져옴
func1(x, y) = (x[0] + y, x[1] + 1)
# x = (1, 1), y = 2
# 계산: (1 + 2, 1 + 1) → (3, 2)
>> 최종 (3,2)

p2: 파티션 2 처리: [3, 4]
1) 초기값: zeroValue = [0,0]
x=(0,0), y = 3 ([3,4] 에 1)

2) 첫번째 요소 3 처리
func1(x, y) = (x[0] + y, x[1] + 1)
# x = (0, 0), y = 3
# 계산: (0 + 3, 0 + 1) → (3, 1)

3) 첫번째 요소 4 처리 # 누적 (3,1)을 가져옴
func1(x, y) = (x[0] + y, x[1] + 1)
# x = (3, 1), y = 4
# 계산: (3 + 4, 1 + 1) → (7, 2)
>> 최종 (7,2)


p1, p2 파티션 결합 
func2(x, y) = (x[0] + y[0], x[1] + y[1])
# x = (3, 2), y = (7, 2)
# 계산: (3 + 7, 2 + 2) → (10, 4)

>> (10,4)

## key-value RDD(Paired RDD)

groupByKey(), mapValues() <br> 
reduceByKey() <br> 
countByKey(), keys(), values() <br>



그룹핑 한 후에 특정 트랜스포메이션을 수행, 키가 필요함. 


In [26]:
rdd9 = spark.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5),
    ("짬뽕", 20),
])
rdd9.collect()

[('짜장면', 15), ('짬뽕', 10), ('짜장면', 5), ('짬뽕', 20)]

In [27]:
#groupByKey()
rdd9_g = rdd9.groupByKey()
rdd9_g.collect()

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x7f577cc4fc70>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x7f577cbf7520>)]

In [28]:
rdd9_g.mapValues(len).collect() #len 대신 max, min 가능 

[('짜장면', 2), ('짬뽕', 2)]

In [29]:
rdd9_g.mapValues(list).collect() 

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [30]:
rdd9_g.mapValues(lambda x:sum(x)/len(x)).collect() #집계형 연산들

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [31]:
#reduceByKey()
rdd9.reduceByKey(add).collect() # add 대신 max,min 쓸수있음, len은 안됨

[('짜장면', 20), ('짬뽕', 30)]

In [32]:
rdd9.reduceByKey(lambda x,y: x+y).collect() 

[('짜장면', 20), ('짬뽕', 30)]

In [33]:
#countByKey() 각 키 별로 요소들의 갯수를 카운트 > Action
rdd9.countByKey()

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

In [34]:
#keys() 키만 가진 RDD > Transformation
rdd9.keys().collect()

['짜장면', '짬뽕', '짜장면', '짬뽕']

In [35]:
#values() valud만 가진 RDD > Transformation
rdd9.values().collect()

[15, 10, 5, 20]

## join - outer, inner - transformation

inner-join : 두개의 집합에서 함께 존재하는 요소(key)의 집합 (Default) <br>
outer-join: 한쪽에있고, 다른쪽에 없을 때 - leftOuterJoin, rightOuterJoin, fullOuterJoin

In [36]:
join_rdd1 = spark.parallelize(
[
    ("a",10),
    ("b",20),
    ("c",30),
])

join_rdd2 = spark.parallelize(
[
    ("a",20),
    ("b",20),
    ("b",10),
    ("d",5),
])

In [37]:
#join - innerjoin, default
join_rdd1.join(join_rdd2).collect() #key를 기준으로 같이 있는 걸 추출 - c는 2번째에 없으니 추출이 안됨

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 20))]

In [38]:
#leftOuterJoin
join_rdd1.leftOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('c', (30, None)), ('a', (10, 20))]

In [39]:
#rightOuterJoin
join_rdd1.rightOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('d', (None, 5)), ('a', (10, 20))]

In [40]:
#fullOuterJoin
join_rdd1.fullOuterJoin(join_rdd2).collect()

[('b', (20, 20)),
 ('b', (20, 10)),
 ('c', (30, None)),
 ('d', (None, 5)),
 ('a', (10, 20))]

In [41]:
spark.stop()